# A Tutorial for the phi_generation Library: Data-Driven Prompting and RAG (Direct Function Calls)

## Introduction

This tutorial demonstrates the key features of the `phi_generation` library, focusing on data ingestion, preparing data for a Retrieval-Augmented Generation (RAG) pipeline, and querying the RAG database with optional CSV context.

The library aims to provide focused and relevant answers from language models by grounding them in specific data.

## Data

Best practices for encoding structured patient data is still under consideration.  The current working practice of the lab is to store patient data in .csv or dataframe format, with alphanumeric entries for identifying patient data, and binary values for whether a patient record mentions each disease, mediation, and symptom of interest.  Because of HIPAA restrictions, "real" data was constructed manually in the current working format, compatible with the Bayesian patient data package also under development in the lab.

**Note:** This tutorial assumes you have the `phi_generation` library installed.

 After making changes to the code, you need to rebuild and reinstall the package.
 Run the following commands on command line in your `phi_generation` project root:

In [ ]:
python -m build
pip install dist/phi_gen-0.1.0.tar.gz --upgrade

SyntaxError: invalid syntax (2719185373.py, line 1)

## 1. Preparing Data from CSV for RAG

This section demonstrates how to convert structured data in a CSV file into a Markdown format suitable for the RAG database using the `utils` module.

In [ ]:
import os
from phi_gen import utils as ut

# Define the path to your CSV file
csv_file_path = r"C:\Users\noliv\Downloads\structured_data_filled.csv"  

# Convert the CSV to a Markdown table
markdown_table = ut.csv_to_markdown_table(csv_file_path)

Some error handling on the above:

In [ ]:
if isinstance(markdown_table, str) and markdown_table.startswith("Error"):
    print(markdown_table)
else:
    print("Successfully converted CSV to Markdown table.")
    print("\n--- Markdown Table Preview ---")
    print(markdown_table[:500] + "...\n--- End Preview ---") # Preview a part of the table

    # Save the Markdown table to the data directory
    save_message = ut.var_markdown_to_data_dir(markdown_table, filename="from_csv_data.md")
    print(save_message)

    # Copy the generated Markdown file to the RAG data directory
    # Assuming your notebook is in the project root
    local_md_path = "phi_gen/rag_module/data/from_csv_data.md"
    ut.local_markdown_to_data_dir(local_md_path)

Successfully converted CSV to Markdown table.

--- Markdown Table Preview ---
| Patient Name   |   Patient Age |   Alzheimers |   Anxiety |   Arthritis |   Behavior |   Bipolar |   Cannabis |   Cardio |   Chronic Disease |   Depression |   Diabetes |   Dieting |   Disabilities |   Drug-Induced Delirium |   Exercise |   Gastrointestinal |   Getting Worse or Not Better |   Hospital Admission |   Hospital Readmission |   Hypertension |   Kidney Disease |   Long-Term Care |   Memory Care |   Mental Health Questionnaire |   Obesity/Metabolic |   Osteoarthritis |   Pain |   Pre...
--- End Preview ---
Markdown written to c:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\..\code_files/rag_module/data\from_csv_data.md


## 2. Adding Additional Documents to the Data Directory (Optional)

You can add more Markdown files to the 'data' directory, which will be processed by the database creation step.

In [3]:
# Example 1: Adding content directly
document_content = """
# This is an example document

This document contains some additional information for the RAG database.
"""
ut.var_markdown_to_data_dir(document_content, filename = "sample_text.md")
print("Added example_document.md to the data directory.")

Added example_document.md to the data directory.


In [ ]:
# Example 2: Copying a local Markdown file

# You can replace with the path to your local .md file, or add many files in a path at once with this function

# This is not an .md so it will throw an error
local_file_to_copy = r"C:\Users\noliv\Downloads\s41592-022-01728-4.pdf" 
copy_result = ut.local_markdown_to_data_dir(local_file_to_copy, "phi_gen/data", enforce_md=True)
print(copy_result)

Error: Only files with the '.md' extension can be copied.


## 3. Creating the RAG Database

Now that we have our data in the 'data' directory (including the Markdown generated from the CSV and any additional documents), we'll create the Chroma vector database using the `create_database` module.

In [ ]:
from phi_gen.rag_module import create_database as crd

# Run the database generation process
crd.generate_data_store()
print("\nRAG database generation complete.")

--- Generating Data Store ---


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Loaded 5 documents.
Split documents into 100 chunks.
Error saving to Chroma: [WinError 32] The process cannot access the file because it is being used by another process: 'chroma\\daee5f4c-9b22-493e-b188-e19f8df24101\\data_level0.bin'
--- Data Store Generation Complete ---

RAG database generation complete.


## 4. Querying the RAG Database with Optional CSV Context (Direct Function Call)

This section demonstrates querying the RAG database using the `query_rag_with_csv` function from the `query_data` module.


This is an unhelpful query, and we expect the response to indicate as such.

In [ ]:
from phi_gen.rag_module import query_data as qd

# Define the query text
query = "Answer the question based on the information provided."

In [7]:
# Option A: Query without providing a CSV file
print("\n--- Querying without CSV Context ---")
response_no_csv = qd.query_rag_with_csv(query)
print(response_no_csv)


--- Querying without CSV Context ---


c:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\rag_module\query_data.py:47: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
c:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\rag_module\query_data.py:60: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = model.predict(prompt)


Response: I'm sorry, but there is no specific question provided to answer based on the context and CSV data. Please provide a question for me to answer.
Sources: ['C:\\Users\\noliv\\phi_generation_repo_destination\\phi_generation\\code_files\\rag_module\\data\\connor_soap_therapy.md', 'C:\\Users\\noliv\\phi_generation_repo_destination\\phi_generation\\code_files\\rag_module\\data\\connor_soap_therapy.md', 'C:\\Users\\noliv\\phi_generation_repo_destination\\phi_generation\\code_files\\rag_module\\data\\connor_soap_therapy.md']


In [8]:

# Option B: Query with a CSV file providing additional context (Direct Function Call)
print("\n--- Querying with CSV Context ---")

# Replace below with your CSV path:
better_query = "Create a patient profile for Kathryn in SOAP format using the structured patient data provided for Kathryn in the .csv file attached, as well as style supplements for writing a SOAP note in the database. Do not list features of patient, but rather explain in sentences."
csv_file_for_query = r"C:\Users\noliv\Downloads\structured_data_filled.csv" 
response_with_csv = qd.query_rag_with_csv(better_query, csv_file_for_query)
print(response_with_csv)


--- Querying with CSV Context ---
Response: Subjective: Kathryn is a 26-year-old female with a history of anxiety, arthritis, and behavioral changes. She denies any current issues with depression, diabetes, or dieting.

Objective: Kathryn presents today with moderate anxiety, mild arthritis symptoms, and some behavioral changes.

Assessment: Patient has a mix of physical and mental health concerns, mainly focused on anxiety and arthritis. Monitoring of these conditions is essential for ongoing care.

Plan: Recommend further evaluation of arthritis symptoms and consider treatment options for anxiety management. Follow up in 2 weeks to assess response to interventions.
Sources: ['C:\\Users\\noliv\\phi_generation_repo_destination\\phi_generation\\code_files\\rag_module\\data\\meta_report_apso_soap.md', 'C:\\Users\\noliv\\phi_generation_repo_destination\\phi_generation\\code_files\\rag_module\\data\\meta_report_apso_soap.md', 'C:\\Users\\noliv\\phi_generation_repo_destination\\phi_generat

This looks promising, and it even drew from the style guide without metadata indicating it was a style guide (there are options for this in the functions that add to database), but note that subjective and objective are the same. Future tasks include to adjust context window, try with various temperatures (and observe error propagation), and improve prompting.

## 5. Comparing Embeddings (Optional : Direct Function Call for Error Handling)

The `compare_embeddings` module allows you to query the database directly and see the retrieved documents along with their relevance scores.

In [ ]:
from phi_gen.rag_module import compare_embeddings as ce
print("\n--- Comparing Embeddings ---")
query_for_comparison = "What are some key details?"
embedding_comparison_results = ce.compare_embeddings_query(query_for_comparison)
print(embedding_comparison_results)


--- Comparing Embeddings ---
--- Matching Results ---
Score: 0.7201408587167824
Content: Plan:
Source: C:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\rag_module\data\connor_soap_therapy.md
---
Score: 0.7193233152227922
Content: Guiding Principles
Source: C:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\rag_module\data\meta_report_apso_soap.md
---
Score: 0.710408245238374
Content: Objective:
Source: C:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\rag_module\data\connor_soap_therapy.md
---
--- Query Ended ---



## 6. Generating Structured to Unstructured PHI

This section uses the `st_to_unst` module to query a vanilla LLM (i.e., without RAG) to create a doctor's record from a .csv file containing structured PHI for one or more patients. For more specific formatting instructions, such as record templates, compare results qualitatively and with the telephone module. 

This section demonstrates how to use the `st_to_unst` module to:
1. Generate patient health records from structured CSV data.
2. Create an AnnData object to store both structured and unstructured data.
3. Convert the unstructured data to a JSON file.

If you do not readily have csv data, you can use the following example csv:

In [ ]:
# ```csv
# PatientID,Name,Age,Gender,ConditionA,ConditionB,MedicationX,MedicationY
# 1,Alice,30,Female,0,1,1,0
# 2,Bob,25,Male,1,0,0,1
# ```

In [ ]:
from phi_gen.st_to_unst_module import st_to_unst as sun
import pandas as pd

# Define the path to your patient data CSV file
patient_csv_file = r"C:\Users\noliv\Downloads\structured_data_filled.csv" # Replace with the actual path


# 1. Generate an AnnData object
print("\n--- Creating AnnData Object ---")
try:
    adata = sun.create_anndata_from_csv(patient_csv_file, patient_id_columns=["PatientID", "Name"])
    print(adata)
    print("\nSample patient records:")
    print(adata.obs.head())
    print("\nSample unstructured data:")
    print(adata.X.head())
except FileNotFoundError:
    print(f"Error: CSV file not found at {patient_csv_file}")
except Exception as e:
    print(f"An error occurred: {e}")


--- Creating AnnData Object ---


c:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\st_to_unst_module\st_to_unst.py:41: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)  # You can adjust temperature
c:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\st_to_unst_module\st_to_unst.py:48: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  report = llm.predict(introduction_prompt)


AnnData object with n_obs × n_vars = 7 × 2
    obs: 'Patient Name', 'Patient Age', 'Alzheimers', 'Anxiety', 'Arthritis', 'Behavior', 'Bipolar', 'Cannabis', 'Cardio', 'Chronic Disease', 'Depression', 'Diabetes', 'Dieting', 'Disabilities', 'Drug-Induced Delirium', 'Exercise', 'Gastrointestinal', 'Getting Worse or Not Better', 'Hospital Admission', 'Hospital Readmission', 'Hypertension', 'Kidney Disease', 'Long-Term Care', 'Memory Care', 'Mental Health Questionnaire', 'Obesity/Metabolic', 'Osteoarthritis', 'Pain', 'Prediabetes', 'Quality of Life', 'Semaglutide', 'Sleep', 'Stress'
    uns: 'csv_filename'

Sample patient records:
  Patient Name Patient Age Alzheimers Anxiety Arthritis Behavior Bipolar  \
0      Kathryn          26          0       1       N/A        1       0   
1      Michael          38          0       0         0        0       0   
2        Kevin          17          0       1         0        0       0   
3        Megan          25          0       1         0        

c:\Users\noliv\phi_generation_repo_destination\phi_generation\venv_ftw_1\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
from phi_gen.st_to_unst_module import st_to_unst as sun
import pandas as pd

# Define the path to your patient data CSV file
patient_csv_file = r"C:\Users\noliv\Downloads\structured_data_filled.csv" # Replace with the actual path



print("\n--- Creating AnnData Object from CSV ---")
try:
    adata = sun.create_anndata_from_csv(patient_csv_file, patient_id_columns=["PatientID", "Name"])

    # Explore the AnnData object
    print(adata)  # Print basic information
    print("\nSample of structured patient data (adata.obs):")
    print(adata.obs.head())  # Show the first few rows of the original CSV data

    print("\nSample of unstructured patient reports (adata.X):")
    if isinstance(adata.X, pd.DataFrame):
        print(adata.X.head())
    else:
        print("adata.X is not a DataFrame. Showing raw data:")
        print(adata.X)  # Or print(adata.X.toarray()) if it's a sparse matrix

except FileNotFoundError:
    print(f"Error: CSV file not found at {patient_csv_file}")
except Exception as e:
    print(f"An error occurred: {e}")


--- Creating AnnData Object from CSV ---


c:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\st_to_unst_module\st_to_unst.py:412: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)  # You can adjust temperature
c:\Users\noliv\phi_generation_repo_destination\phi_generation\code_files\st_to_unst_module\st_to_unst.py:419: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  report = llm.predict(introduction_prompt)


AnnData object with n_obs × n_vars = 7 × 2
    obs: 'Patient Name', 'Patient Age', 'Alzheimers', 'Anxiety', 'Arthritis', 'Behavior', 'Bipolar', 'Cannabis', 'Cardio', 'Chronic Disease', 'Depression', 'Diabetes', 'Dieting', 'Disabilities', 'Drug-Induced Delirium', 'Exercise', 'Gastrointestinal', 'Getting Worse or Not Better', 'Hospital Admission', 'Hospital Readmission', 'Hypertension', 'Kidney Disease', 'Long-Term Care', 'Memory Care', 'Mental Health Questionnaire', 'Obesity/Metabolic', 'Osteoarthritis', 'Pain', 'Prediabetes', 'Quality of Life', 'Semaglutide', 'Sleep', 'Stress'
    uns: 'csv_filename'

Sample of structured patient data (adata.obs):
  Patient Name Patient Age Alzheimers Anxiety Arthritis Behavior Bipolar  \
0      Kathryn          26          0       1       N/A        1       0   
1      Michael          38          0       0         0        0       0   
2        Kevin          17          0       1         0        0       0   
3        Megan          25          0   

In [3]:
print(adata.X)

[['N/A, N/A'
  "\nIntroduction\n\nPatient Name: John Smith\nAge: 45\nGender: Male\nAssigned Number: 12345\n\nAs a doctor, it is important to regularly assess the overall health of patients in order to prevent and manage any potential illnesses. In this case, Mr. John Smith, a 45-year-old male, has presented for a routine check-up. Upon reviewing his medical records, it is noted that he has a history of several health conditions and is currently taking multiple medications. A thorough evaluation of his health is necessary to ensure that he is receiving appropriate care and treatment for his existing conditions.\n\nHypertension and Diabetes\n\nMr. Smith has been diagnosed with both hypertension and diabetes, two chronic conditions that require ongoing management and monitoring. Hypertension, also known as high blood pressure, is a condition in which the force of blood against the artery walls is too high. This can put strain on the heart and lead to serious health complications if left u

In [ ]:
import csv

csv_file_path =  r"C:\Users\noliv\Downloads\structured_data_filled.csv" # Replace with your actual path

try:
    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        patient_data = list(reader)

    print("\n--- Loaded CSV Data ---")
    for row in patient_data:
        print(row)
    print("--- End Loaded CSV Data ---")

except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
except Exception as e:
    print(f"An error occurred while loading CSV: {e}")


--- Loaded CSV Data ---
{'Patient Name': 'Kathryn', 'Patient Age': '26', 'Alzheimers': '0', 'Anxiety': '1', 'Arthritis': 'N/A', 'Behavior': '1', 'Bipolar': '0', 'Cannabis': '1', 'Cardio': '1', 'Chronic Disease': '0', 'Depression': 'N/A', 'Diabetes': '0', 'Dieting': '0', 'Disabilities': '0', 'Drug-Induced Delirium': '0', 'Exercise': '0', 'Gastrointestinal': '0', 'Getting Worse or Not Better': '0', 'Hospital Admission': '0', 'Hospital Readmission': '0', 'Hypertension': '0', 'Kidney Disease': '0', 'Long-Term Care': '0', 'Memory Care': '0', 'Mental Health Questionnaire': '0', 'Obesity/Metabolic': '0', 'Osteoarthritis': '0', 'Pain': '0', 'Prediabetes': '0', 'Quality of Life': '0', 'Semaglutide': '0', 'Sleep': '0', 'Stress': '0'}
{'Patient Name': 'Michael', 'Patient Age': '38', 'Alzheimers': '0', 'Anxiety': '0', 'Arthritis': '0', 'Behavior': '0', 'Bipolar': '0', 'Cannabis': '0', 'Cardio': '1', 'Chronic Disease': '0', 'Depression': '0', 'Diabetes': '0', 'Dieting': '0', 'Disabilities': '0', '

In [ ]:
import csv

csv_file_path = r"C:\Users\noliv\Downloads\structured_data_filled.csv"  # Replace with your actual path

try:
    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        patient_data = list(reader)

    print("\n--- Loaded CSV Data ---")
    for row in patient_data:
        print(row)
    print("--- End Loaded CSV Data ---")

except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
except Exception as e:
    print(f"An error occurred while loading CSV: {e}")


--- Loaded CSV Data ---
{'Patient Name': 'Kathryn', 'Patient Age': '26', 'Alzheimers': '0', 'Anxiety': '1', 'Arthritis': 'N/A', 'Behavior': '1', 'Bipolar': '0', 'Cannabis': '1', 'Cardio': '1', 'Chronic Disease': '0', 'Depression': 'N/A', 'Diabetes': '0', 'Dieting': '0', 'Disabilities': '0', 'Drug-Induced Delirium': '0', 'Exercise': '0', 'Gastrointestinal': '0', 'Getting Worse or Not Better': '0', 'Hospital Admission': '0', 'Hospital Readmission': '0', 'Hypertension': '0', 'Kidney Disease': '0', 'Long-Term Care': '0', 'Memory Care': '0', 'Mental Health Questionnaire': '0', 'Obesity/Metabolic': '0', 'Osteoarthritis': '0', 'Pain': '0', 'Prediabetes': '0', 'Quality of Life': '0', 'Semaglutide': '0', 'Sleep': '0', 'Stress': '0'}
{'Patient Name': 'Michael', 'Patient Age': '38', 'Alzheimers': '0', 'Anxiety': '0', 'Arthritis': '0', 'Behavior': '0', 'Bipolar': '0', 'Cannabis': '0', 'Cardio': '1', 'Chronic Disease': '0', 'Depression': '0', 'Diabetes': '0', 'Dieting': '0', 'Disabilities': '0', '

## Conclusion

This tutorial demonstrated how to use the `phi_generation` library to ingest data from CSV files and local Markdown files, build a RAG database, and query it with the option to include CSV data directly in the prompt. The `utils` module provides helpful functions for data handling, and the direct function calls offer a more integrated experience within the Jupyter Notebook. 

In [ ]:
from phi_gen.st_to_unst_module import st_to_unst as sun
import pandas as pd

patient_csv_file = r"C:\Users\noliv\Downloads\structured_data_filled.csv"

print("\n--- Creating AnnData Object from CSV ---")
try:
    adata = sun.process_csv_and_create_anndata(
        patient_csv_file,
        patient_id_columns=["Patient Name"]  # <-- fixed here
    )

    print(adata)
    print("\nSample of structured patient data (adata.obs):")
    print(adata.obs.head())

    print("\nSample of unstructured patient reports (adata.X):")
    print(pd.DataFrame(adata.X, columns=["Patient Name", "Doctor's Note"]).head())

except FileNotFoundError:
    print(f"Error: CSV file not found at {patient_csv_file}")
except Exception as e:
    print(f"An error occurred: {e}")



--- Creating AnnData Object from CSV ---
AnnData object with n_obs × n_vars = 7 × 2
    obs: 'Patient Name', 'Patient Age', 'Alzheimers', 'Anxiety', 'Arthritis', 'Behavior', 'Bipolar', 'Cannabis', 'Cardio', 'Chronic Disease', 'Depression', 'Diabetes', 'Dieting', 'Disabilities', 'Drug-Induced Delirium', 'Exercise', 'Gastrointestinal', 'Getting Worse or Not Better', 'Hospital Admission', 'Hospital Readmission', 'Hypertension', 'Kidney Disease', 'Long-Term Care', 'Memory Care', 'Mental Health Questionnaire', 'Obesity/Metabolic', 'Osteoarthritis', 'Pain', 'Prediabetes', 'Quality of Life', 'Semaglutide', 'Sleep', 'Stress'
    uns: 'csv_filename'

Sample of structured patient data (adata.obs):
  Patient Name  Patient Age  Alzheimers  Anxiety  Arthritis  Behavior  \
0      Kathryn           26           0        1        NaN         1   
1      Michael           38           0        0        0.0         0   
2        Kevin           17           0        1        0.0         0   
3        M

In [2]:
!python -m unittest discover -s tests



.........
----------------------------------------------------------------------
Ran 9 tests in 58.187s

OK
